In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.preprocessing import minmax_scale
import random
import cv2
from imgaug import augmenters as iaa
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import keras
from keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
inceptionres = load_model('../input/cassavamodels/model (1).h5')
res50 = load_model('../input/cassavamodels/model (2).h5')


In [ ]:
sample = pd.read_csv(os.path.join('../input/cassava-leaf-disease-classification/sample_submission.csv'))
print(sample)

In [ ]:
TARGET_SIZE = 512
test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator= test_datagen.flow_from_dataframe(sample,
                         directory = os.path.join('../input/cassava-leaf-disease-classification/test_images'),
                         x_col = "image_id",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         class_mode=None)
pred1 = inceptionres.predict_generator(test_generator, verbose=1)
pred2 = res50.predict_generator(test_generator, verbose=1)

In [ ]:
preds=[]
for i in range(0,len(pred1)):
    preds.append((pred1[i]+pred2[i])/2)

In [ ]:
final = []
for x in preds:
    final.append(np.argmax(x))
submission = pd.DataFrame({ 'image_id': sample.image_id, 'label': final })
submission.to_csv("submission.csv", index=False)

In [ ]:
print(submission)